# Project Roadmap


### Taking Measurements of ONE file

1.	Import libraries
2.	Read txt file and extract timestamps, phonemes
3.	Define functions that measure formants and pitch
4.  Create a .csv file and Data Frame containing all data from text file and measurements


### Extract all files that need to be analysed

1.  List all files in a directory
2.  Select directory : audio/text & smile/neutral
3.  Create dictionary for each label
4.  Full list of all files for a speaker


### Create full database of measurements for a speaker
1.  Define parameters needed for each measurement
2.  Iterate through filename list and measure
3.  Create and populate database




## Taking Measurements of ONE file

### 1. Import Libraries

In [1]:
import csv
import pandas as pd
import parselmouth 
from parselmouth import praat
from parselmouth.praat import call
import os


### 2. Transform text (.lab) files with timestamps into a csv file with aggregated data

#### STEPS

- 2.1 extract info from text file and put it in a list
- 2.2 clean data: keep only vowels, transform to ms unit
- 2.3 enrich data: add vowel length and medial point for future measurements, file name and smile/neutral label
- 2.4 order columns and give titles
- 2.5 create csv file with measurements for each token

In [2]:
def text_to_csv(new_csv,textfile,directory):
    
    #2.1 create a list where all the info read from the text file will be stored
    stamps = []
    with open(directory+textfile,"r") as text:
        lines = text.readlines()
        for line in lines:
            stamps.append(line.split())

   
    #2.2 clean data
          
    #delete silences
    for i in stamps:
        if i[2] == 'sil':
            del stamps[stamps.index(i)]
            
            
    #delete consonants
    for i in stamps:
        phoneme = i[2]
        
        v = any(vowel in phoneme for vowel in ['a','e','i','o','u'])
        
        if v==True:
            pass
        else:
            del stamps[stamps.index(i)]
    
    
        
    #transform into integers and ms    
    for i in stamps:
        
        i[0] = int(i[0])/10000000
        i[1] = int(i[1])/10000000
    
    

    #2.3 enrich data
    
    for i in stamps:
        
        #add time of measurement, the medial time of the vowel 
        middle = ((i[1]-i[0])/2)+i[0]
        i.append(middle)
    
        #calculate vowel length
        i.append(i[1]-i[0])
        
        #add file name 
        i.append(textfile[:-4])
        
        #add smile/neutral tag
        if "smile" in directory:
            i.append('smile')
        elif "neutral" in directory:
            i.append('neutral')
            
            
    #2.4 order
    order = [6, 2, 3, 4, 0, 1, 5]
    text_data = []
    
    for i in stamps:
        i = [i[x] for x in order]
        text_data.append(i)
        
    
    
    #2.5 write csv with enriched data from text file and headers
    
    with open(new_csv+".csv","w") as f:
        write = csv.writer(f)
        write.writerow(["Label","Phoneme","Measure Time","Vowel Length","Initial Time", "End Time", "File"])
        write.writerows(text_data)
        
    
    return text_data

### 3. Define function to measure Formant and Pitch

#### The following articles and documentation were used for this part:

- Formant extraction tutorial: http://blog.syntheticspeech.de/2021/03/10/how-to-extract-formant-tracks-with-praat-and-python/

- Parselmouth documentation: https://parselmouth.readthedocs.io/en/stable/

- Praat scripting documentation: https://www.fon.hum.uva.nl/praat/manual/Scripting.html


In [3]:
def measure_formants(audio_directory, text_data):
    
    # Define variables needed, according Parselmouth documentation
        #the audio file we will analyse
    sound = parselmouth.Sound(audio_directory) 
    
        #call to use Praat function to measure
    formants = call(sound, "To Formant (burg)", 0.0025, 5, 5000, 0.025, 50)

    
    # Start empty lists for measurement of formants
    f1_list = []
    f2_list = []
    f3_list = []

    
    #make a measurement for each phoneme (item in the list)
    for i in text_data:
    
        t = i[2]     #time of the measurement, the medial point calculated above
        
        #measure each formant at this time and add to the list of measurements for this audio file
        f1 = call(formants, "Get value at time", 1, t, 'Hertz', 'Linear')
        f2 = call(formants, "Get value at time", 2, t, 'Hertz', 'Linear')
        f3 = call(formants, "Get value at time", 3, t, 'Hertz', 'Linear')
    
        f1_list.append(float(f1))
        f2_list.append(float(f2))
        f3_list.append(float(f3))
        
    
    #Return as a tupple for easy unpacking
    
    return f1_list,f2_list,f3_list



In [4]:
def measure_pitch(audio_directory,text_data):
    #same structure as previous function, only with a different Praat function
    
    sound = parselmouth.Sound(audio_directory) 
    pitch = call(sound, "To Pitch", 0.0, 25, 500) 
    
    pitch_list = []

    for i in text_data:
    
        t = i[2]
        get_pitch = call(pitch, "Get value at time",t, 'Hertz', 'Linear')
    
        pitch_list.append(get_pitch)
        
    
    
    return pitch_list


In [5]:
#add all measurements into a single list

def add_measurements(testfile, text_data):
    
    (f1, f2, f3) = measure_formants(testfile,text_data)
    f0 = measure_pitch(testfile,text_data)
        
    
    for i in text_data:
        i.append(f0[text_data.index(i)])
        i.append(f1[text_data.index(i)])
        i.append(f2[text_data.index(i)])
        i.append(f3[text_data.index(i)])
        
    return text_data




### 4. Create a .csv file and Data Frame containing all data from text file and measurements

In [6]:
def complete_csv (full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name):
    
    # use function from step 2 to extract data from text file
    text_data = text_to_csv(complete_csv_name, text_file_name, text_file_dir)
    
    # use function from step 3 to make measurements
    complete_data = add_measurements(full_audio_dir, text_data)
    
    #write a new csv file containing both sets of data
    
    with open(complete_csv_name+".csv","w") as f:
        write = csv.writer(f)
        write.writerows(complete_data)
        
    return complete_data
    

In [8]:
#turn csv to DataFrame for future manipulation and analysis

def complete_df (full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name):
    
    data = complete_csv(full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name)
    headers = ["Label","Phoneme","Measure Time","Vowel Length","Initial Time", "End Time", "File", "f0","f1","f2","f3"]

    df = pd.DataFrame(data, columns=headers)
    
    df = df[~df.Phoneme.isin(['a','e','i','o','u'])]
    
    return df
    
    

In [9]:
#Example + list of the parameters I need to obtain the final DataFrame of a single pair of files (audio/text)

full_audio_dir = '/Users/julietaloizaga/OneDrive - Queen Mary, University of London/Documents/Coding4Lin/Final Project/smile/wav/SpkBeng_0004.wav'

initial_csv_name = "testfile1"

text_file_name = "SpkBeng_0001.lab"

text_file_dir = "smile/lab/"

complete_csv_name = text_file_name[:-4]+"-full_data"

In [10]:
complete_df (full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name)

,Label,Phoneme,Measure Time,Vowel Length,Initial Time,End Time,File,f0,f1,f2,f3
0,smile,sil,0.150,0.12,0.09,0.21,SpkBeng_0001,NaN,1108.720833,2430.263398,3154.525378
1,smile,ao,0.245,0.07,0.21,0.28,SpkBeng_0001,171.706818,372.053929,985.843977,2248.084681
2,smile,er,0.485,0.13,0.42,0.55,SpkBeng_0001,160.476852,482.263705,1273.106720,2096.421304
3,smile,ah,0.570,0.04,0.55,0.59,SpkBeng_0001,154.451144,252.929876,1408.954000,2564.039439
4,smile,dh,0.670,0.06,0.64,0.70,SpkBeng_0001,NaN,1446.514425,2398.407619,3658.167067
5,smile,ax,0.735,0.07,0.70,0.77,SpkBeng_0001,NaN,1069.228480,1827.071658,2340.587419
6,smile,ey,0.910,0.12,0.85,0.97,SpkBeng_0001,NaN,1187.439240,2187.476691,3118.313477
7,smile,jh,1.065,0.07,1.03,1.10,SpkBeng_0001,159.620044,446.028084,1253.225107,2400.755428
8,smile,er,1.150,0.10,1.10,1.20,SpkBeng_0001,177.377997,520.081207,1347.829371,2489.429565
9,smile,r,1.370,0.06,1.34,1.40,SpkBeng_0001,170.671384,451.394139,1225.420255,2151.447413


## Extract all files that need to be analysed

In [10]:
#get a list of the names of all files in a directory
def get_filenames(folder):

    directory = os.fsencode(folder)
    file_name_list = []
    
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        file_name_list.append(filename)
    
    return file_name_list

In [11]:
#select directory
def get_directory(label, medium, general_dir):
    folder = general_dir
    if label == 'smile':
        folder += "smile/"
    else:
        folder += "neutral/"
    if medium == 'audio':
        folder += 'wav/'
    else:
        folder+= 'lab/'
        
    return folder

In [12]:
#create a dictionary with key = text file and value = audio file 

def file_dict(list_of_filename_lists, label):

    if label == "smile":
        text_list = list_of_filename_lists[1]
        audio_list = list_of_filename_lists[0]
    else:
        text_list = list_of_filename_lists[3]
        audio_list = list_of_filename_lists[2]
    
    label_dict = {}
    

    for i in text_list:
        label_dict[i] = audio_list[text_list.index(i)]
        
    return label_dict    
    
    
#sort lists to make sure audio001 matches index with text001

def sortlists(lists):
    for i in lists:
        i.sort()
        
    return lists



In [13]:
#create a single dictionary for a speaker containing 2 dictionaries (smile/neutral), each containg pairs of audio and text files

def get_speaker_lists(general_dir):
    
    #use previous function to create 4 lists smile/neutral audio/text
    smile_audio_list = get_filenames(get_directory('smile','audio',general_dir))
    smile_text_list = get_filenames(get_directory('smile','text',general_dir))
    neutral_audio_list = get_filenames(get_directory('neutral','audio',general_dir))
    neutral_text_list = get_filenames(get_directory('neutral','text',general_dir))

    #join in one big list and sort them with previous function
    list_of_filename_lists = [smile_audio_list, smile_text_list, neutral_audio_list, neutral_text_list]

    sortlists(list_of_filename_lists)
    
    #create 2 dictiona
    neutral_dict = file_dict(list_of_filename_lists,"neutral")
    smile_dict = file_dict(list_of_filename_lists,"smile")
    
    speaker_dict = {"neutral":neutral_dict,"smile":neutral_dict}
    
    return speaker_dict

## Create full database of measurements for a speaker

In [14]:
#Restating the Variables I need for measuring each pair of files
full_audio_dir = ""
file_name = ""
initial_csv_name = file_name+"-initial_data"
complete_csv_name = file_name+"-full_data"
text_file_name = ""
text_file_dir = ""

In [15]:
def get_neutral (general_dir):
    measurements = []
    
    neutral_dict = get_speaker_lists(general_dir)["neutral"]
    for i in neutral_dict:
        full_audio_dir = general_dir+"neutral/wav/"+neutral_dict[i]
        file_name = (neutral_dict[i])[:-3]
        initial_csv_name = file_name+"-initial_data"
        complete_csv_name = file_name+"-full_data"
        text_file_name = i
        text_file_dir = general_dir+"neutral/lab/"
        
        measurement = complete_df (full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name)   
        measurements.append(measurement)
    
    return measurements

In [16]:
def get_smile(general_dir):
    measurements = []
    
    smile_dict = get_speaker_lists(general_dir)["smile"]
    for i in smile_dict:
        full_audio_dir = general_dir+"smile/wav/"+smile_dict[i]
        file_name = (smile_dict[i])[:-3]
        initial_csv_name = file_name+"-initial_data"
        complete_csv_name = file_name+"-full_data"
        text_file_name = i
        text_file_dir = general_dir+"smile/lab/"
        
        measurement = complete_df (full_audio_dir, initial_csv_name, text_file_name, text_file_dir, complete_csv_name)   
        measurements.append(measurement)
    
    return measurements

In [18]:
def get_general_dir():
    return str(input("What is the directory where you have placed the 'Smile' and 'Neutral' folders? Please input the full directory:"))


def speaker_data(general_dir=get_general_dir()):
    
    neutral_df = get_neutral(general_dir).DataFrame()
    smile_df = get_smile(general_dir).DataFrame()
    
    all_speaker_data = [neutral_df, smile_df]
    
    return pd.concat(all_speaker_data, keys=["neutral","smile"])
    

What is the directory where you have placed the 'Smile' and 'Neutral' folders? Please input the full directory:/Users/julietaloizaga/OneDrive - Queen Mary, University of London/Documents/Coding4Lin/Final Project/


In [20]:
speaker_data()

AttributeError: 'list' object has no attribute 'DataFrame'

What is the directory where you have placed the 'Smile' and 'Neutral' folders? Please input the full directory:/Users/julietaloizaga/OneDrive - Queen Mary, University of London/Documents/Coding4Lin/Final Project/


'/Users/julietaloizaga/OneDrive - Queen Mary, University of London/Documents/Coding4Lin/Final Project/'